<a href="https://colab.research.google.com/github/ricky-kiva/dl-tensorflow-intro/blob/main/3_a1_mnist_convolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Convolution on MNIST dataset**

In [ ]:
# import libraries
import tensorflow as tf
import numpy as np

Load dataset

In [ ]:
# instantiate dataset from tensorflow API
mnist = tf.keras.datasets.mnist

# get only train data
(training_images, training_labels), _ = mnist.load_data()

# see shape
training_images.shape

(60000, 28, 28)

Make function to reshape & normalize train image data

In [ ]:
def reshape_and_normalize(images):

  # reshape the image to add extra dimension (for color)
  images = images.reshape(60000, 28, 28, 1)

  # normalize images
  images = images / 255.0

  return images

Do & check reshape and normalize

In [ ]:
training_images = reshape_and_normalize(training_images)

print(f'Maximum pixel value after normalization: {np.max(training_images)}\n')
print(f'Shape of training set after reshape: {training_images.shape}\n')
print(f'Shape of single image after reshape: {training_images[0].shape}')

Maximum pixel value after normalization: 1.0

Shape of training set after reshape: (60000, 28, 28, 1)

Shape of single image after reshape: (28, 28, 1)


Defining callback

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.995):
      print('\nAccuracy is higher than 99.5%, cancelling the training!')
      self.model.stop_training = True

Build the model

In [ ]:
def convolutional_model():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D((2,2)),

      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

  return model

Train the model

In [ ]:
# save the model to variable
model = convolutional_model()

# get number of weights
model_params = model.count_params()

# unit test to limit the size model the model
assert model_params < 1000000, (
    f'Your model has {model.params:,} params. '
    f'Keep it under 1,000,000 by reducing number of units in Conv2D and/or Dense layers'
)

# instantiate callback class
callbacks = myCallback()

# train model
history = model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])

Epoch 1/10
1875/1875 [==============================] - 62s 32ms/step - loss: 0.1452 - accuracy: 0.9566
Epoch 2/10
1875/1875 [==============================] - 48s 25ms/step - loss: 0.0485 - accuracy: 0.9850
Epoch 3/10
1875/1875 [==============================] - 46s 24ms/step - loss: 0.0315 - accuracy: 0.9900
Epoch 4/10
1875/1875 [==============================] - 47s 25ms/step - loss: 0.0209 - accuracy: 0.9934
Epoch 5/10
1874/1875 [============================>.] - ETA: 0s - loss: 0.0133 - accuracy: 0.9956
Accuracy is higher than 99.5%, cancelling the training!
1875/1875 [==============================] - 48s 26ms/step - loss: 0.0133 - accuracy: 0.9956


In [ ]:
# print epoch information
print(f'Your model was trained for {len(history.epoch)} epochs')